# Question 1.1


In [ ]:
import gensim.downloader

w2v = gensim.downloader.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
w2v.save_word2vec_format('word2vec-google-news-300.bin', binary=True)

Question 1.1

In [ ]:
similar_to_student = w2v.most_similar('student', topn=1)[0]
print(f"Similar word to 'student' is '{similar_to_student[0]}' with  cosine similarity: {similar_to_student[1]}")

similar_to_Apple = w2v.most_similar('Apple', topn=1)[0]
print(f"Similar word to 'Apple' is '{similar_to_Apple[0]}' with  cosine similarity: {similar_to_Apple[1]}")

similar_to_apple = w2v.most_similar('apple', topn=1)[0]
print(f"Similar word to 'apple' is '{similar_to_apple[0]}' with  cosine similarity: {similar_to_apple[1]}")

Similar word to 'student' is 'students' with  cosine similarity: 0.7294867038726807
Similar word to 'Apple' is 'Apple_AAPL' with  cosine similarity: 0.7456986308097839
Similar word to 'apple' is 'apples' with  cosine similarity: 0.720359742641449


Question 1.2 a)


In [ ]:
import pandas as pd
from collections import Counter

train_data = pd.read_csv("eng.train", sep=' ', header=None, names=['word', 'pos', 'chunk', 'label'], skip_blank_lines=False)
dev_data = pd.read_csv("eng.testa", sep=' ', header=None, names=['word', 'pos', 'chunk', 'label'], skip_blank_lines=False)
test_data = pd.read_csv("eng.testb", sep=' ', header=None, names=['word', 'pos', 'chunk', 'label'], skip_blank_lines=False)

# Function to get number of sentences and unique labels
def get_info(data):
    num_sentences = data['word'].isnull().sum()
    labels = data['label'].dropna().tolist()
    unique_labels = set(labels)
    return num_sentences, unique_labels

# Get information
train_info = get_info(train_data)
dev_info = get_info(dev_data)
test_info = get_info(test_data)

print(f'Training set: {train_info[0]} sentences, Labels: {train_info[1]}')
print(f'Development set: {dev_info[0]} sentences, Labels: {dev_info[1]}')
print(f'Test set: {test_info[0]} sentences, Labels: {test_info[1]}')

Training set: 14989 sentences, Labels: {'I-ORG', 'I-LOC', 'B-MISC', 'B-LOC', 'I-PER', 'I-MISC', 'B-ORG', 'O'}
Development set: 3468 sentences, Labels: {'O', 'I-LOC', 'B-MISC', 'I-PER', 'I-MISC', 'I-ORG'}
Test set: 3683 sentences, Labels: {'O', 'I-LOC', 'B-MISC', 'B-LOC', 'I-PER', 'I-MISC', 'B-ORG', 'I-ORG'}


1.2b)

In [ ]:
from collections import Counter
import pandas as pd

def find_example_sentence(data):
    current_sentence = []
    current_entity = ""
    current_label = None
    multi_word_entities = Counter()
    for index, row in data.iterrows():
        if pd.isnull(row['word']):
            # Sentence boundary
            if len(multi_word_entities) >= 2:
                return current_sentence
            # Reset for next sentence
            current_sentence = []
            multi_word_entities = Counter()
        else:
            current_sentence.append(row)
            prefix, label = row['label'].split('-') if '-' in row['label'] else (None, None)
            if prefix == 'B' or (prefix == 'I' and label != current_label):
                # New entity
                if current_entity.count(' ') >= 1:  # Check if the entity has more than one word
                    multi_word_entities[current_label] += 1
                current_entity = row['word']
                current_label = label
            elif prefix == 'I':
                # Continuation of the current entity
                current_entity += ' ' + row['word']

# Load data
train_data = pd.read_csv('eng.train', sep=' ', header=None, names=['word', 'pos', 'chunk', 'label'], skip_blank_lines=False)

# Find and print example sentence
example_sentence = find_example_sentence(train_data)
example_sentence_df = pd.DataFrame(example_sentence)
print(example_sentence_df[['word', 'label']])

              word  label
47         Germany  I-LOC
48              's      O
49  representative      O
50              to      O
51             the      O
52        European  I-ORG
53           Union  I-ORG
54              's      O
55      veterinary      O
56       committee      O
57          Werner  I-PER
58       Zwingmann  I-PER
59            said      O
60              on      O
61       Wednesday      O
62       consumers      O
63          should      O
64             buy      O
65       sheepmeat      O
66            from      O
67       countries      O
68           other      O
69            than      O
70         Britain  I-LOC
71           until      O
72             the      O
73      scientific      O
74          advice      O
75             was      O
76         clearer      O
77               .      O


In [ ]:
def list_named_entities(sentence):
    named_entities = []
    current_entity = []
    current_label = None
    for _, row in sentence.iterrows():
        label_parts = row['label'].split('-')
        prefix, label = label_parts if len(label_parts) > 1 else (None, None)
        if prefix == 'B' or (prefix == 'I' and label != current_label):
            # New named entity
            if current_entity:
                named_entities.append((' '.join(current_entity), current_label))
            current_entity = [row['word']]
            current_label = label
        elif prefix == 'I':
            # Continuation of current named entity
            current_entity.append(row['word'])
    # Don't forget the last named entity
    if current_entity:
        named_entities.append((' '.join(current_entity), current_label))
    return named_entities

named_entities = list_named_entities(example_sentence_df)
for entity, label in named_entities:
    print(f'{entity} ({label})')

Germany (LOC)
European Union (ORG)
Werner Zwingmann (PER)
Britain (LOC)


1.3a)

In [ ]:
!pip install seqeval

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from seqeval.metrics import f1_score
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import time
from sklearn.preprocessing import LabelEncoder

In [ ]:
def handle_oov_words(word, surrounding_words, embeddings):
    global zero_vector_count  # for counter of zero vector
    global average_embedding_count # for embeeding use count

    if word in embeddings:
        return embeddings[word]

    elif surrounding_words:
        surrounding_embeddings = [embeddings[w] for w in surrounding_words if w in embeddings]
        if surrounding_embeddings:
            # computing average
            avg_embedding = np.mean(surrounding_embeddings, axis=0)
            average_embedding_count += 1
            return avg_embedding
    # only return zero vector if otherwise
    zero_vector_count += 1
    return np.zeros(embeddings.vector_size)

def get_embedding_for_sentence(sentence, embeddings):
    sentence_embeddings = []
    for i, word in enumerate(sentence):
        surrounding_words = sentence[max(0, i-2):i] + sentence[i+1:min(len(sentence), i+3)]
        word_embedding = handle_oov_words(word, surrounding_words, embeddings)
        sentence_embeddings.append(word_embedding)
    return sentence_embeddings

In [ ]:
sentences_list = []
labels_list = []
current_sentence = []
current_labels = []

zero_vector_count = 0
average_embedding_count = 0

with open('eng.train', 'r') as file:
    for line in file:
        # Split lines into words and labels
        tokens = line.strip().split()
        if tokens:  # If line is not blank
            word, pos, chunk, label = tokens
            current_sentence.append(word)
            current_labels.append(label)
        else:  # If line is blank (end of sentence)
            if current_sentence and current_labels:  # If there's a sentence/labels to add
                sentences_list.append(current_sentence)
                labels_list.append(current_labels)
                # Reset current_sentence and current_labels for the next sentence
                current_sentence = []
                current_labels = []

    # code for opening file gotten from chatgpt

if current_sentence and current_labels:
    sentences_list.append(current_sentence)
    labels_list.append(current_labels)

sentence_embeddings = [get_embedding_for_sentence(sentence, w2v) for sentence in sentences_list]

label_encoder = LabelEncoder()
encoded_labels = [label_encoder.fit_transform(labels) for labels in labels_list]

print(f'Number of zero vectors used: {zero_vector_count}')
print(f'Number of average embeddings used: {average_embedding_count}')

Number of zero vectors used: 4339
Number of average embeddings used: 52176


In [ ]:
def load_conll_data(filename):
    data = pd.read_csv(filename, sep=' ', header=None, names=['word', 'pos', 'chunk', 'label'], skip_blank_lines=False)
    sentences = []
    sentence = []
    for index, row in data.iterrows():
        if pd.isnull(row['word']):
            if len(sentence) > 0:
                sentences.append(sentence)
            sentence = []
        else:
            word = row['word']
            label = row['label']
            if word in w2v:
                sentence.append((word, label))
    return sentences

train_sentences = load_conll_data('eng.train')
dev_sentences = load_conll_data('eng.testa')
test_sentences = load_conll_data('eng.testb')

# Create vocabulary and label sets
word_vocab = set(word for sentence in train_sentences + dev_sentences + test_sentences for word, _ in sentence)
label_set = set(label for sentence in train_sentences + dev_sentences + test_sentences for _, label in sentence)

# Map words and labels to indices
word_to_index = {word: i for i, word in enumerate(word_vocab)}
label_to_index = {label: i for i, label in enumerate(label_set)}
index_to_label = {i: label for label, i in label_to_index.items()}

# Define a function to convert a sentence to input and target tensors
def sentence_to_tensors(sentence):
    words, labels = zip(*sentence)
    word_indices = torch.tensor([word_to_index[word] for word in words], dtype=torch.long)
    label_indices = torch.tensor([label_to_index[label] for label in labels], dtype=torch.long)
    return word_indices, label_indices

# Convert sentences to tensors
train_data = [sentence_to_tensors(sentence) for sentence in train_sentences]
dev_data = [sentence_to_tensors(sentence) for sentence in dev_sentences]
test_data = [sentence_to_tensors(sentence) for sentence in test_sentences]

# Define the NER model
class NERModel(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, label_size, pretrained_embeddings):
        super(NERModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding.weight.data.copy_(pretrained_embeddings)
        self.embedding.weight.requires_grad = False  # Freeze the embeddings
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2, label_size)

    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)
        logits = self.fc(lstm_out)
        return logits

# Create word embeddings matrix
embedding_dim = 300
vocab_size = len(word_vocab)
embedding_matrix = torch.zeros(vocab_size, embedding_dim)
for word, index in word_to_index.items():
    if word in w2v:
        embedding_matrix[index] = torch.from_numpy(w2v[word])

# Instantiate the NER model
hidden_dim = 256
label_size = len(label_set)
ner_model = NERModel(embedding_dim, hidden_dim, vocab_size, label_size, embedding_matrix)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(ner_model.parameters())

# Define a function to train the NER model
def train_ner_model(model, train_data, dev_data, optimizer, criterion, num_epochs=10):
    total_start_time = time.time()
    for epoch in range(num_epochs):
        epoch_start_time = time.time()
        model.train()
        total_loss = 0
        for word_indices, label_indices in train_data:
            optimizer.zero_grad()
            outputs = model(word_indices.unsqueeze(0))  # Add a batch dimension
            labels = label_indices.unsqueeze(0)  # Add a batch dimension
            loss = criterion(outputs.view(-1, label_size), labels.view(-1))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(train_data)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}')

        epoch_end_time = time.time()
        epoch_time = epoch_end_time - epoch_start_time
        print(f'Time taken for epoch {epoch+1}: {epoch_time:.2f} seconds')

        # Evaluate on the development set
        model.eval()
        predictions = []
        true_labels = []
        with torch.no_grad():
            for word_indices, label_indices in dev_data:
                outputs = model(word_indices.unsqueeze(0))
                _, predicted = torch.max(outputs, 2)
                predicted = predicted.squeeze(0).tolist()
                true_labels.append([index_to_label[i] for i in label_indices.tolist()])  # Convert indices to labels
                predictions.append([index_to_label[i] for i in predicted])
        f1 = f1_score(true_labels, predictions)
        print(f'F1 Score: {f1:.4f}')

        # Check for early stopping
        if epoch > 0 and f1 <= max_f1:
            print('Early stopping...')
            break
        max_f1 = f1
    total_end_time = time.time()
    total_time = total_end_time - total_start_time
    print(f'Total time taken for all epochs: {total_time:.2f} seconds')

# Train the NER model
train_ner_model(ner_model, train_data, dev_data, optimizer, criterion, num_epochs=10)



<ipython-input-11-d18cd1630131>:64: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  embedding_matrix[index] = torch.from_numpy(w2v[word])


Epoch 1/10, Loss: 0.1524
Time taken for epoch 1: 237.11 seconds
F1 Score: 0.8497
Epoch 2/10, Loss: 0.0690
Time taken for epoch 2: 227.16 seconds
F1 Score: 0.8832
Epoch 3/10, Loss: 0.0375
Time taken for epoch 3: 236.30 seconds
F1 Score: 0.8959
Epoch 4/10, Loss: 0.0230
Time taken for epoch 4: 238.22 seconds
F1 Score: 0.8967
Epoch 5/10, Loss: 0.0176
Time taken for epoch 5: 237.60 seconds
F1 Score: 0.8958
Early stopping...
Total time taken for all epochs: 1227.53 seconds


In [ ]:
# Define a function to predict labels for a sentence
def predict_labels(model, sentence):
    model.eval()
    with torch.no_grad():
        word_indices, _ = sentence_to_tensors(sentence)
        outputs = model(word_indices.unsqueeze(0))
        _, predicted = torch.max(outputs, 2)
        predicted = predicted.squeeze(0).tolist()
        labels = [index_to_label[i] for i in predicted]
        return labels

# Test the NER model on a sentence
example_sentence = train_sentences[0]
predicted_labels = predict_labels(ner_model, example_sentence)
print('Example Sentence:')
print(' '.join([word for word, _ in example_sentence]))
print('Predicted Labels:')
print(predicted_labels)

Example Sentence:
EU rejects German call boycott British lamb
Predicted Labels:
['I-ORG', 'O', 'I-MISC', 'O', 'O', 'I-MISC', 'O']


In [ ]:
from seqeval.metrics import f1_score

# Prepare predicted and ground truth labels
predicted_labels = [predict_labels(ner_model, sentence) for sentence in test_sentences]
true_labels = [[label for _, label in sentence] for sentence in test_sentences]

# Calculate F1 score
f1 = f1_score(true_labels, predicted_labels)
print(f'F1 Score: {f1:.2f}')


F1 Score: 0.85


In [ ]:
# #Q2

# from random import randint
# train = pd.read_csv('train.csv', usecols=["label-coarse", "text"])
# test = pd.read_csv('test.csv', usecols=["label-coarse", "text"])
# merge1 = randint(0,5)
# merge2 = randint(0,5)
# while merge1==merge2:
#     merge2 = randint(0,5)
# train = train.replace([merge1,merge2],"OTHERS")
# test = test.replace([merge1,merge2],"OTHERS")
# dev = train.sample(n=500)
# train = train.drop(dev.index)


# Part 2

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import time

# For reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Load the datasets
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Randomly select 4 unique classes from the 6 different coarse labels
selected_classes = np.random.choice(train_df['label-coarse'].unique(), 4, replace=False)
# Ensure selected_classes is a list of strings
selected_classes = [str(cls) for cls in selected_classes]
print(f"Selected classes: {selected_classes}")

# Make sure the 'label-coarse' column is of string type before comparison
train_df['label-coarse'] = train_df['label-coarse'].astype(str)
test_df['label-coarse'] = test_df['label-coarse'].astype(str)

# Apply the lambda function to update the labels
train_df['label-coarse'] = train_df['label-coarse'].apply(lambda x: x if x in selected_classes else 'OTHERS')
test_df['label-coarse'] = test_df['label-coarse'].apply(lambda x: x if x in selected_classes else 'OTHERS')

# Now, the label_encoder should work without type error
label_encoder = LabelEncoder()
labels_train = label_encoder.fit_transform(train_df['label-coarse'].values)
labels_test = label_encoder.transform(test_df['label-coarse'].values)

# Preprocessing function for text
def preprocess_text(sentence):
    words = sentence.lower().split()  # simple tokenization and lowercasing
    # Remove stop words, punctuation, etc. if necessary
    return words

# Function to convert sentences to a matrix of word embeddings
def sentence_to_avg(sentence, w2v_model):
    words = preprocess_text(sentence)
    embedding_matrix = [w2v_model[word] for word in words if word in w2v_model]
    if not embedding_matrix:  # Handle sentences without any words in vocabulary
        return np.zeros(w2v_model.vector_size)
    avg_embedding = np.mean(embedding_matrix, axis=0)
    return avg_embedding

# Preprocess and encode the labels
label_encoder = LabelEncoder()
labels_train = label_encoder.fit_transform(train_df['label-coarse'].values)
labels_test = label_encoder.transform(test_df['label-coarse'].values)

# Convert sentences to average embeddings
X_train = np.array([sentence_to_avg(sentence, w2v) for sentence in train_df['text'].values])
X_test = np.array([sentence_to_avg(sentence, w2v) for sentence in test_df['text'].values])

# Split the training data into training and development sets
X_train, X_dev, y_train, y_dev = train_test_split(X_train, labels_train, test_size=500/len(train_df), random_state=42)

# Convert numpy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_dev_tensor = torch.tensor(X_dev, dtype=torch.float)
y_dev_tensor = torch.tensor(y_dev, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float)
y_test_tensor = torch.tensor(labels_test, dtype=torch.long)

# Create TensorDatasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
dev_dataset = TensorDataset(X_dev_tensor, y_dev_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create DataLoaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


Selected classes: ['0', '1', '5', '2']


In [ ]:
# Function to convert sentences to a matrix of word embeddings
def sentence_to_sum(sentence, w2v_model):
    words = preprocess_text(sentence)
    embedding_matrix = [w2v_model[word] for word in words if word in w2v_model]
    if not embedding_matrix:  # Handle sentences without any words in vocabulary
        return np.zeros(w2v_model.vector_size)
    avg_embedding = np.sum(embedding_matrix, axis=0)
    return avg_embedding

In [ ]:
# Function to convert sentences to a matrix of word embeddings
def sentence_to_max(sentence, w2v_model):
    words = preprocess_text(sentence)
    embedding_matrix = [w2v_model[word] for word in words if word in w2v_model]
    if not embedding_matrix:  # Handle sentences without any words in vocabulary
        return np.zeros(w2v_model.vector_size)
    avg_embedding = np.max(embedding_matrix, axis=0)
    return avg_embedding

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import random

# Define the model with dropout for regularization
class QuestionClassifier(nn.Module):
    def __init__(self, input_size, output_size, hidden_size, dropout_rate=0.5):
        super(QuestionClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = x.float()  # make sure x is float
        out = self.dropout(self.relu(self.fc1(x)))
        out = self.dropout(self.relu(self.fc2(out)))
        out = self.fc3(out)
        return self.softmax(out)

# Assuming `embedding_matrix` is of shape (vocab_size, embedding_dim)
# and `vocab_size` is the total number of words and `embedding_dim` is the dimension of the word vectors.
embedding_dim = w2v.vector_size  # This should be the size of your word vectors
# output_size = len(np.unique(y_train))  # This should be the number of unique classes
output_size = 5  # This should now be 5 since there are 5 classes including 'OTHERS'
hidden_size = 128  # You can tune this
dropout_rate = 0.5  # Regularization with dropout

# Initialize the model
model = QuestionClassifier(input_size=embedding_dim, output_size=output_size, hidden_size=hidden_size, dropout_rate=dropout_rate)

# Define the loss function and the optimizer with L2 regularization (weight decay)
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)  # Added weight decay for L2 regularization

# Add early stopping logic based on development set accuracy
best_dev_acc = 0.0
epochs_no_improve = 0
n_epochs_stop = 3  # number of epochs with no improvement after which training will be stopped

# Training loop
num_epochs = 20  # You may want to use early stopping based on dev set performance
total_start_time = time.time()
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Evaluate on development set
    model.eval()
    with torch.no_grad():
        correct, total = 0, 0
        for inputs, labels in dev_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        accuracy = correct / total
        print(f'Epoch {epoch + 1}, Dev Accuracy: {accuracy:.4f}')

        # Early stopping logic
    if accuracy > best_dev_acc:
        best_dev_acc = accuracy
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        if epochs_no_improve == n_epochs_stop and epoch!=num_epochs-1:
            print("Early stopping")
            break
total_end_time = time.time()
# Evaluate on test set
model.eval()
with torch.no_grad():
    correct, total = 0, 0
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    accuracy = correct / total
    print(f'Test Accuracy: {accuracy:.4f}')
    print("Time taken:",total_end_time-total_start_time)


Epoch 1, Dev Accuracy: 0.6240
Epoch 2, Dev Accuracy: 0.6960
Epoch 3, Dev Accuracy: 0.7220
Epoch 4, Dev Accuracy: 0.7380
Epoch 5, Dev Accuracy: 0.7320
Epoch 6, Dev Accuracy: 0.7600
Epoch 7, Dev Accuracy: 0.7500
Epoch 8, Dev Accuracy: 0.7640
Epoch 9, Dev Accuracy: 0.7700
Epoch 10, Dev Accuracy: 0.7740
Epoch 11, Dev Accuracy: 0.7840
Epoch 12, Dev Accuracy: 0.7940
Epoch 13, Dev Accuracy: 0.7960
Epoch 14, Dev Accuracy: 0.8040
Epoch 15, Dev Accuracy: 0.7720
Epoch 16, Dev Accuracy: 0.8000
Epoch 17, Dev Accuracy: 0.8140
Epoch 18, Dev Accuracy: 0.8160
Epoch 19, Dev Accuracy: 0.8100
Epoch 20, Dev Accuracy: 0.8120
Test Accuracy: 0.8620
Time taken: 16.036020278930664
